## Redispatch data of wind power plants in Langenhorn (Schleswig-Holstein)

In [1]:
# packages
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

**Redipatch data of the location Langenhorn (not only wind)**

Website Redispatch data SH-Netz (DSO): https://www.sh-netz.com/de/energie-einspeisen/redispatch-2-0/einspeisemanagement/veroeffentlichungen/abgeschlossene-massnahmen.html

In [2]:
df_redispatch = pd.read_csv("../data/Redispatch_Langenhorn.csv", sep = ';')

columns_to_keep = ['Start', 'Ende', 'Dauer (Min)', 'Ort Engpass', 
                   'Stufe (%)', 'Ursache', 'Anlagenschlüssel'] 
                   #'Einsatz-ID', , 'Gebiet', 'Anforderer', 'Netzbetreiber',
                   #'Anlagen-ID', 'Entschädigungspflicht']

column_name_mapping = {
#    'Einsatz-ID': 'mission_ID',
    'Start': 'start_redispatch',
    'Ende': 'end_redispatch',
    'Dauer (Min)': 'duration (min)',
#    'Gebiet': 'location',
    'Ort Engpass': 'congestion_location',
    'Stufe (%)': 'level',
    'Ursache': 'cause',
    'Anlagenschlüssel': 'plant_key_eeg',
#    'Anforderer': 'requester',
#    'Netzbetreiber': 'grid_operator',
#    'Anlagen-ID': 'plant_ID',
#    'Entschädigungspflicht': 'compensation_obligation'
}

# Rename columns
df_redispatch = df_redispatch[columns_to_keep].rename(columns=column_name_mapping)

# change data types of columns 
df_redispatch['start_redispatch'] = pd.to_datetime(df_redispatch['start_redispatch'], errors='coerce', format='mixed', dayfirst=True)
df_redispatch['end_redispatch'] = pd.to_datetime(df_redispatch['end_redispatch'], errors='coerce', format='mixed', dayfirst=True)

**EEG-keys and plant type data of Langenhorn (not only wind)**

Website:
http://www.energymap.info/energieregionen/DE/105/119/477/19932.html

In [3]:
# load data
df_eeg = pd.read_csv("../data/EEG_Langenhorn.csv", sep = ';')

# select relevant features
columns_to_keep = ['Anlagenschluessel', 'Anlagentyp', 'Inbetriebnahme', 'DSO', 'TSO', 
                   #'Nennleistung(kWp_el)', 'kWh(2013)', 'kWh(average)', 'kWh/kW', 
                   'GPS-Lat', 'GPS-Lon']

# translate column names
column_name_mapping = {
    'Inbetriebnahme': 'commissioning_date',
    'Anlagenschluessel': 'plant_key_eeg',
    'Anlagentyp': 'plant_type',
#    'Nennleistung(kWp_el)': 'rated_capacity_kWp', # bruttoleistung
    'DSO': 'dso',
    'TSO': 'tso',
#    'kWh(2013)': 'kWh_2013',
#    'kWh(average)': 'avg_kWh',
#    'kWh/kW': 'kWh/kW',
    'GPS-Lat': 'lat',
    'GPS-Lon': 'long'
}

df_eeg = df_eeg[columns_to_keep].rename(columns=column_name_mapping)

# change data types of columns 
df_eeg['commissioning_date'] = pd.to_datetime(df_eeg['commissioning_date'], errors='coerce', format='mixed', dayfirst=True)
#df_eeg['rated_capacity_kWp'] = df_eeg['rated_capacity_kWp'].str.replace(',', '.')
#df_eeg['kWh_2013'] = df_eeg['kWh_2013'].str.replace('.', '')
#df_eeg['avg_kWh'] = df_eeg['avg_kWh'].str.replace('.', '')
#df_eeg['avg_kWh'] = df_eeg['avg_kWh'].str.replace(',', '.')
df_eeg['lat'] = df_eeg['lat'].str.replace(',', '.')
df_eeg['long'] = df_eeg['long'].str.replace(',', '.')
#df_eeg['kWh_2013'] = df_eeg['kWh_2013'].astype(float)
#df_eeg['rated_capacity_kWp'] = df_eeg['rated_capacity_kWp'].astype(float)
#df_eeg['avg_kWh'] = df_eeg['avg_kWh'].astype(float)
df_eeg['lat'] = df_eeg['lat'].astype(float)
df_eeg['long'] = df_eeg['long'].astype(float)

**Filter wind power plants in df_redispatch by mapping EEG-keys of wind plants from df_eeg**

In [4]:
df_redispatch_wind = pd.merge(df_redispatch, df_eeg[df_eeg['plant_type'] == 'Windkraft'], on='plant_key_eeg', how='inner')


In [5]:
df_redispatch_wind.columns

Index(['start_redispatch', 'end_redispatch', 'duration (min)',
       'congestion_location', 'level', 'cause', 'plant_key_eeg', 'plant_type',
       'commissioning_date', 'dso', 'tso', 'lat', 'long'],
      dtype='object')

In [6]:
df_redispatch_wind['plant_key_eeg'].unique() # 12 unique wind plants 

array(['E20793012S12X00000000002414080001',
       'E20793012S12Z00000000002414080001',
       'E20793012S12000000000002414080001',
       'E20793012S12Y00000000002414080001',
       'E20793012S12V00000000002414080001',
       'E20793012S12W00000000002414080001',
       'E2079301EA01000000000070577400001',
       'E2079301EA01000000000070577400002',
       'E2079301EA01000000000070577400003',
       'E2079301EA01000000000070577400004',
       'E2079301EA01000000000070577400005',
       'E2079301EA01000000000070577400006'], dtype=object)

**Look up if the 12 identified wind plants are registered in the Marktstammdatenregister**

Website:
https://www.marktstammdatenregister.de/MaStR/Einheit/Einheiten/OeffentlicheEinheitenuebersicht

In [7]:
# Marktstammdatenregister Windkraft Langenhorn
df_register = pd.read_csv("../data/Marktstammdatenregister_Langenhorn.csv", sep = ';')

# select relevant features
columns_to_keep = ['MaStR-Nr. der Einheit', 'Inbetriebnahmedatum der Einheit',
                   #'Bruttoleistung der Einheit', 'Nettonennleistung der Einheit', '\tMaStR-Nr. des Anlagenbetreibers',
                   'Name des Anlagenbetreibers (nur Org.)']

# translate column names
column_name_mapping = {
    'MaStR-Nr. der Einheit': 'plant_key_mastr',
    'Inbetriebnahmedatum der Einheit': 'commissioning_date',
#    'Bruttoleistung der Einheit': 'gross_capacity',
#    'Nettonennleistung der Einheit': 'net_capacity',
#    '\tMaStR-Nr. des Anlagenbetreibers': 'operator_key',
    'Name des Anlagenbetreibers (nur Org.)': 'operator_name'
}

df_register = df_register[columns_to_keep].rename(columns=column_name_mapping)
df_register['commissioning_date'] = pd.to_datetime(df_register['commissioning_date'], errors='coerce', format='mixed', dayfirst=True)

In [8]:
# Merge the DataFrames on 'commissioning_date'
df_redispatch_wind_registered = pd.merge(df_redispatch_wind, df_register, on='commissioning_date', how='inner')

**Create a df minute-by-minute redispatch of the 11 wind plants found in the register**

In [9]:
# remove duplicated
df_redispatch_wind_registered = df_redispatch_wind_registered.drop_duplicates()
df_redispatch_wind_registered.reset_index(inplace=True, drop=True)

In [10]:
columns_to_remove = ['duration (min)', 'commissioning_date']
df_redispatch_wind_registered.drop(columns_to_remove, axis = 1, inplace = True)

In [11]:
# redispatch of the last two years
latest_end = df_redispatch_wind_registered['end_redispatch'].max()
earliest_start = latest_end - timedelta(days=365 * 2)

In [ ]:
time_index = pd.date_range(start=earliest_start, end=latest_end, freq='T')

def update_redispatch_status(row):
    return (row['start_redispatch'] <= time_index) & (time_index <= row['end_redispatch'])

redispatch_status = df_redispatch_wind_registered.apply(lambda row: update_redispatch_status(row), axis=1)
other_columns = df_redispatch_wind_registered.apply(lambda col: col.values.repeat(len(time_index)), axis=0)

wind_redispatch = pd.DataFrame({'redispatch': redispatch_status}, index=time_index)
wind_redispatch.reset_index(inplace=True)
wind_redispatch['index'] = wind_redispatch['index'].dt.floor('min')
wind_redispatch.set_index('index', inplace=True)

wind_redispatch[list(other_columns.index)] = other_columns.T
wind_redispatch['redispatch'] = wind_redispatch['redispatch'].astype(int)

In [ ]:
new_order = ['redispatch', 'level', 'cause', 'plant_type', 'plant_key_eeg', 'plant_key_mastr',
             'lat', 'long', 'operator_name', 'congestion_location', 'dso', 'tso']

wind_redispatch = wind_redispatch.reindex(columns=new_order)

In [ ]:
wind_redispatch.head()

In [ ]:
# store csv
wind_redispatch.to_csv('wind_redispatch_minute.csv',sep = ',')